Copyright 2023 Borislav Rumenov Varbanov

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

In [ ]:
import requests
import pandas as pd
from requests.auth import HTTPBasicAuth
import json
import calendar
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import zscore
from prophet import Prophet
import re

debug_bool = False

# Load configuration from JSON file
def load_config():
    with open('config.json', 'r') as f:
        return json.load(f)

config = load_config()
if debug_bool:
    print("Config dict print:")
    for enum, value in config.items():
        print("  ", enum, " : ", value)

In [ ]:
BASE_URL = "https://api.octopus.energy"

def fetch_tariff_data(tariff_code):
    TARIFF_URL = f"{BASE_URL}/v1/products/{config['PRODUCT_CODE']}/{tariff_code}/standard-unit-rates/"
    PARAMS = {
        "period_from": config['PERIOD_FROM'],
        "period_to": config['PERIOD_TO'],
        "page_size": config['PAGE_SIZE']
    }
    response = requests.get(TARIFF_URL, params=PARAMS, auth=HTTPBasicAuth(config['API_KEY'], ""))
    
    if response.status_code != 200:
        print(f"Error fetching data for {tariff_code}: {response.status_code} - {response.text}")
        return None
    else:
        return response.json()

all_data = []

for utility, tariff_code in config['TARIFF_CODE'].items():
    if debug_bool:
        print(tariff_code)
    tariff_data = fetch_tariff_data(tariff_code)
    
    if tariff_data:
        data = []
        for result in tariff_data['results']:
            valid_from = result['valid_from']
            valid_to = result['valid_to']
            value_inc_vat = result['value_inc_vat']
            if debug_bool:
                print(f"Price between {valid_from} and {valid_to} for {tariff_code} is {value_inc_vat} p/kWh")
            data.append([tariff_code, valid_from, valid_to, value_inc_vat])
        
        all_data.extend(data)

if all_data:
    df = pd.DataFrame(all_data, columns=['Tariff Code', 'Valid From', 'Valid To', 'Price (p/kWh)'])
    csv_file = "tariff_data.csv"
    df.to_csv(csv_file, index=False)
    if debug_bool:
        print(f"Data saved to {csv_file}")
else:
    if debug_bool:
        print("No data to process.")

In [ ]:
df = pd.DataFrame(all_data, columns=['Tariff Code', 'Valid From', 'Valid To', 'Price (p/kWh)'])

df['Valid From'] = pd.to_datetime(df['Valid From'])
df['Valid To'] = pd.to_datetime(df['Valid To'])

df['Date'] = df['Valid From'].dt.date
df['Month'] = df['Valid From'].dt.month_name()
monthly_prices = df.groupby(['Tariff Code', 'Month'])['Price (p/kWh)'].mean().reset_index()
# Extract the year
df['Year'] = df['Valid From'].dt.year

# Group by Tariff Code, Year and Month, then calculate the mean price
monthly_prices = df.groupby(['Tariff Code', 'Year', 'Month'])['Price (p/kWh)'].mean().reset_index()

# Sort 'Month' in chronological order
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
monthly_prices['Month'] = pd.Categorical(monthly_prices['Month'], categories=month_order, ordered=True)
monthly_prices = monthly_prices.sort_values(['Year', 'Month'])

# Create interactive bar plot using Plotly
fig = px.bar(
    monthly_prices,
    x='Month',
    y='Price (p/kWh)',
    color='Tariff Code',
    title='Monthly Energy Prices',
    labels={'Month': 'Month', 'Price (p/kWh)': 'Price (p/kWh)'},
    hover_data=['Price (p/kWh)', 'Year'],
    facet_row='Year'  # Separate plot for each year
)

# Update layout
fig.update_layout(
    xaxis_title='Month',
    yaxis_title='Price (p/kWh)',
    hovermode="x unified"
)

# Save as HTML
fig.write_html("energy_month_year_average_plot_interactive.html")

# Show the plot
fig.show()

In [ ]:
# Create a mapping of month names to numbers
month_to_num = {name: num for num, name in enumerate(calendar.month_name) if num}

# Calculate mean and total prices for each Tariff Code
monthly_stats = df.groupby(['Tariff Code', 'Year', 'Month']).agg({'Price (p/kWh)': 'mean'}).reset_index()

# Calculate total mean prices across all Tariff Codes
total_monthly_stats = df.groupby(['Year', 'Month']).agg({'Price (p/kWh)': 'mean'}).reset_index()

# Map month names to numbers in the DataFrames
monthly_stats['Month_num'] = monthly_stats['Month'].map(month_to_num)
total_monthly_stats['Month_num'] = total_monthly_stats['Month'].map(month_to_num)

# Sort by Year and Month_num
monthly_stats = monthly_stats.sort_values(['Year', 'Month_num'])
total_monthly_stats = total_monthly_stats.sort_values(['Year', 'Month_num'])



fig = go.Figure()

# Add bar traces for mean prices by Tariff Code
for tariff_code in monthly_stats['Tariff Code'].unique():
    subset = monthly_stats[monthly_stats['Tariff Code'] == tariff_code]
    for year in subset['Year'].unique():
        yearly_subset = subset[subset['Year'] == year]
        fig.add_trace(go.Bar(x=yearly_subset['Month'], y=yearly_subset['Price (p/kWh)'], 
                             name=f'Mean {tariff_code} {year}', legendgroup=str(year),
                             hoverinfo='x+y'))

# Add line traces for total mean prices across all Tariff Codes
for year in total_monthly_stats['Year'].unique():
    yearly_subset = total_monthly_stats[total_monthly_stats['Year'] == year]
    fig.add_trace(go.Scatter(x=yearly_subset['Month'], y=yearly_subset['Price (p/kWh)'], 
                             mode='lines+markers', name=f'Total Mean {year}', legendgroup=str(year),
                             hoverinfo='x+y'))

# Update layout
fig.update_layout(
    title='Monthly Energy Prices: Mean and Total',
    xaxis_title='Month',
    yaxis_title='Price (p/kWh)',
    hovermode="x unified",
    barmode='group'
)

# Save as HTML
fig.write_html("energy_month_year_mean_total_combined_plot_interactive.html")

# Show the plot
fig.show()

In [ ]:
df['Z-Score'] = df.groupby('Tariff Code')['Price (p/kWh)'].transform(zscore)
anomalies = df[df['Z-Score'].abs() > 2]

# Create line plot using Plotly Graph Objects for more control
fig = go.Figure()

# Add line traces for each tariff code
for tariff_code in df['Tariff Code'].unique():
    subset = df[df['Tariff Code'] == tariff_code]
    fig.add_trace(go.Scatter(x=subset['Valid From'], y=subset['Price (p/kWh)'], mode='lines', name=f'Tariff {tariff_code}'))

# Add scatter points for anomalies with conditional coloring
for i, row in anomalies.iterrows():
    color = 'red' if row['Z-Score'] > 0 else 'green'
    fig.add_trace(go.Scatter(x=[row['Valid From']], y=[row['Price (p/kWh)']], mode='markers', marker=dict(color=color, size=10),
                             name='Anomaly' if i == 0 else None,  # Show legend only once
                             showlegend=(i == 0)))

# Annotate anomalies
for i, row in anomalies.iterrows():
    fig.add_annotation(
        x=row['Valid From'],
        y=row['Price (p/kWh)'],
        text=str(row['Price (p/kWh)']),
        showarrow=True,
        arrowhead=1,
        ax=0,
        ay=-40
    )

# Update layout for better readability
fig.update_layout(
    title='Energy Prices with Anomalies',
    xaxis_title='Date and Time',
    yaxis_title='Price (p/kWh)',
    hovermode="x unified"
)

# Save as HTML
fig.write_html("energy_anomalies_line_plot_interactive_v2.html")

# Show the plot
fig.show()

In [ ]:
# Loop through each unique Tariff Code (assuming TARIFF_CODES contains the codes for gas and electricity)
for utility, tariff_code in config['TARIFF_CODE'].items():
    # Prepare data for Prophet (ds: timestamp, y: value to forecast)
    prices = df[df['Tariff Code'] == tariff_code][['Valid From', 'Price (p/kWh)']]
    prices.columns = ['ds', 'y']
    prices['ds'] = pd.to_datetime(prices['ds']).dt.tz_localize(None)  # Remove timezone

    # Create and fit a Prophet model
    model = Prophet()
    model.fit(prices)

    # Predict future prices for the next 3 months (approximately 90 days)
    future = model.make_future_dataframe(periods=90, freq='D')
    forecast = model.predict(future)

    # Create interactive plot using Plotly
    fig = go.Figure()

    # Add actual price trace
    fig.add_trace(go.Scatter(x=prices['ds'], y=prices['y'], mode='lines', name='Actual'))

    # Add forecast trace
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines', name='Forecast'))

    # Add uncertainty intervals
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat_upper'], fill='tonexty', mode='none', name='Upper Confidence Interval'))
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat_lower'], fill='tonexty', mode='none', name='Lower Confidence Interval'))

    # Update layout
    fig.update_layout(
        title=f'{tariff_code} Price Forecast for Next 3 Months',
        xaxis_title='Date and Time',
        yaxis_title='Price (p/kWh)',
        hovermode="x unified"
    )

    # Sanitize Tariff Code for use in a filename
    sanitized_code = re.sub(r'[\\/*?:"<>|]', '_', tariff_code).lower()
    # Save as HTML
    fig.write_html(f"{sanitized_code.lower()}_prices_3_month_prophet_anomalies_plot_interactive.html")

    # Show the plot
    fig.show()
